In [ ]:
from board_detction import get_squares
from get_position import get_fen_from_board
from GUIPlayer import show_board,choose_color,show_id_manager_interface,ask_to_play_another,setup_realtime_board_view
import matplotlib.pyplot as plt
from comments import explain_move
# from code_Arm import move_piece,cleanup,get_pict

import torch
import matplotlib.pyplot as plt
import pickle
import torch
from torchvision import transforms
from PIL import Image
import chess
import chess.engine
import chess.pgn
import os
import numpy as np
import datetime
import chess
from tensorflow.keras.models import load_model
import tkinter as tk
from tkinter import messagebox
import os

col={x:i for i,x in enumerate(["a","b","c","d","e","f","g","h"])}
row={i+1:i for i in range(9)}

#-------------------------------------------------------------------------------------------------------------------------------------------   


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
COUNTER_FILE = "game_counter.txt"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

engine_path=r"stockfish\stockfish-windows-x86-64-avx2.exe"
engine = chess.engine.SimpleEngine.popen_uci(engine_path)

#loading models--------------------------------------------------------------------------------------------------------------------------

model_for_piece = torch.load('models\chess_piece_classifier.pth')  
model_for_piece = model_for_piece.to(DEVICE)  

# Load the color model
model_for_color = torch.load('models\chess_color_classifier.pth') 
model_for_color = model_for_color.to(DEVICE)  



#---functions deal with changes on the board----------------------------------------------------------------------------------------------------------------------------------------    
def get_best_move(board, time_limit: float = 2.0) -> str:

    with chess.engine.SimpleEngine.popen_uci(engine_path) as engine:
        limit = chess.engine.Limit(time=time_limit)
        
        result = engine.play(board, limit)
        best_move = result.move
    
    return best_move 


def get_moved_piece(predicted_move, board):
    # Start Stockfish engine
    with chess.engine.SimpleEngine.popen_uci(engine_path) as engine:
        # Make the predicted move on the board
        move = chess.Move.from_uci(predicted_move)
        board.push(move)

        # Get the piece that moved
        moved_piece = board.piece_at(move.from_square)

        # Map the piece to a human-readable form
        piece_map = {
            chess.PAWN: "Pawn",
            chess.KNIGHT: "Knight",
            chess.BISHOP: "Bishop",
            chess.ROOK: "Rook",
            chess.QUEEN: "Queen",
            chess.KING: "King"
        }
        
        # Get the piece type that was moved
        moved_piece_type = piece_map.get(moved_piece.piece_type, "Unknown Piece")
        
        return moved_piece_type
    
def get_move(fen, level="beginner"):
    board = chess.Board(fen)
    
    # Set options based on player level
    if level == "beginner":
        options = {
            "Skill Level": 5,          
            "UCI_LimitStrength": True, # Limit engine strength
            "UCI_Elo": 1350           
        }
        time_limit = 0.1  
    elif level == "pro":
        options = {
            "Skill Level": 15,         
            "UCI_LimitStrength": False # Full engine strength
        }
        time_limit = 0.5  

    # Apply the options to Stockfish
    for option, value in options.items():
        engine.configure({option: value})

    # Get the best move within the time limit
    result = engine.play(board, chess.engine.Limit(time=time_limit))
    return result.move


def get_move_from_fen(fen_before, fen_after):
    board_before = chess.Board(fen_before)
    board_after = chess.Board(fen_after)
    
    legal_moves = list(board_before.legal_moves)

    for move in legal_moves:
        board_before.push(move)

        if board_before.board_fen() == board_after.board_fen():
            return move

        board_before.pop()
        
    
    return None  
#---functions of teaching model----------------------------------------------------------------------------------------------------------------------------------------    
   
def evaluate_move( board, move_uci, time=1.0):

    move = chess.Move.from_uci(move_uci)

    if move in board.legal_moves:

        result = engine.analyse(board, chess.engine.Limit(time=time), moves=[move])
        evaluation_score = result["score"].relative.score  # Get evaluation score
        return evaluation_score
    else:
        return None


def load_model_and_tokenizer(id):
    model=None
    tokenizer=None
    if os.path.exists(f"models based on previous games\\players_models\\playerLSTMs{id}.h5"):
        model = load_model(f"models based on previous games\\players_models\\playerLSTMs{id}.h5")


        # Load the tokenizer
        with open(f"models based on previous games\\players_models\\playertokenizer{id}.pkl", "rb") as file:
            tokenizer = pickle.load(file)

    return model,tokenizer 

def lstm_predict(model, tokenizer, sequence):

    sequence_tokenized = tokenizer.texts_to_sequences([sequence])  
    sequence_padded = np.array(sequence_tokenized).reshape(1, -1) 
    

    predictions = model.predict(sequence_padded, verbose=0) 
    predicted_index = np.argmax(predictions)  
    return predicted_index, predictions

def compute_rl_feedback(predicted_move, actual_move,board,c):
    f={0:0.8,1:0.3,2:0.5}
    w1=f[c] if c !=-1 else 0.5
    w2=1-w1 if c !=-1 else 0.5,
    E1=evaluate_move(move_uci=predicted_move,board=board)
    E2=evaluate_move(move_uci=actual_move,board=board)

    S=1 if actual_move==predicted_move else 0

    R=w1*(E2-E1)+w2*S

    improvementFB="Your learning journey is progressing. Even though you're exploring new strategies, keep focusing on the lessons from past games."
    mistakeFB="We can adjust the lessons to match your learning style better. Stay focused and take your time."

    comment= improvementFB if E2>E1 else mistakeFB

    return R,comment


def train_lstm_with_reward(model, tokenizer, sequence, actual_move, reward):
    sequence_padded = np.array(sequence).reshape(1, -1)
    vocab_size = len(tokenizer.word_index) + 1
    y_true = np.zeros((1, vocab_size))

    if actual_move not in tokenizer.word_index:
        tokenizer.word_index[actual_move] = vocab_size
        vocab_size += 1
        y_true = np.zeros((1, vocab_size))
    
    target_index = tokenizer.word_index[actual_move]
    y_true[0, target_index] = reward

    model.fit(sequence_padded, y_true, epochs=1, verbose=0)


  

class ChessGame:
    def __init__(self,level,id,name):
        #,mistakes,miss,brilliants,blunders,forks_and_pins,optimal_moves
        self.board = chess.Board()
        self.active_color = 'White'
        self.playerid=id
        self.playername=name
        self.playerlevel=level
        self.moves = []
        self.result = None
        self.student_color =choose_color()
        self.arm_color =list({"White","Black"}-{self.student_color})[0]
        self.mistakes=0
        self.miss=0
        self.brilliants=0
        self.blunders=0
        self.forks_and_pins=0
        self.optimal_moves=0

        self.middle_game_start = False  # Flag for middle game start

    def update(self, values):
        if len(values) != 6:
            print("Error: The input list must contain exactly 6 values.")
            return
        
        self.mistakes += values[0]
        self.miss += values[1]
        self.brilliants += values[2]
        self.blunders += values[3]
        self.forks_and_pins += values[4]
        self.optimal_moves += values[5]    

    def make_move(self, move):
        if isinstance(move, chess.Move) and move in self.board.legal_moves:
            self.board.push(move)
            self.moves.append(move)   
            return True
        else:
            print(self.board.legal_moves)
            print("Illegal move!")
            return False


    def save_to_pgn(self, filename="game.pgn"):

        
        game = chess.pgn.Game()
        game.headers["Event"] = "TSPY12"
        game.headers["Site"] = "local"
        game.headers["Date"] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        game.headers["White"] = self.playername if self.student_color=="White" else "arm" #id_p
        game.headers["Black"] = self.playername if self.student_color=="Black" else "arm"
        game.headers["id"]=self.playerid
        game.headers["level"]=self.playerlevel
        game.headers["mistakes"] = str(self.mistakes)
        game.headers["blunders"] = str(self.blunders)
        game.headers["forkes_and_pins"] = str(self.forks_and_pins)
        game.headers["optimal_moves"] = str(self.optimal_moves)
        game.headers["birilliants"]=str(self.brilliants)
        game.headers["miss"]=str(self.miss)
        game.headers["Result"] = self.result or "*"


        # Add moves to PGN
        node = game
        for move in self.moves:
            node = node.add_variation(move)

        # Save to file
        with open(filename,"a") as pgn_file:
            print(game, file=pgn_file)
            print("", file=pgn_file)
  
def speak(msg):
    print  
        
def pick_and_place(board, move):

    move_obj = chess.Move.from_uci(move)

    piece = board.piece_at(move_obj.from_square)
    if piece is None:
        return "No piece at the starting square of the move!"
        
    comments = {
            chess.PAWN: f"A pawn's strength lies in promotion. Consider advancing wisely!",
            chess.KNIGHT: f"Knights are best in the center. Remember they move in an 'L' shape.",
            chess.BISHOP: f"Bishops excel in long diagonals. Watch out for obstacles.",
            chess.ROOK: f"Rooks dominate open files. Ensure it stays active!",
            chess.QUEEN: f"The queen is powerful but vulnerable. Don't overexpose it.",
            chess.KING: f"Always protect the king. Don't move it unnecessarily."
        }
    comment = comments.get(piece.piece_type, "Piece-specific strategy applies here.")
        
    best_move = get_best_move(board)
    return f"Move played: {move}. {comment} What about playing {best_move} instead?"

# Example usage
def play(name,id,level,cluster):
    model,tokenizer=load_model_and_tokenizer(id)
    print("Tokenizer Vocabulary:", tokenizer.word_index)

    game = ChessGame(level,id,name)
    fen_before=game.board.fen()

    game_sequence = []
    
    fig, ax = setup_realtime_board_view()
    while not game.board.is_game_over():
        if game.active_color == game.student_color:
            comment=""
            legal_move=False
            while not legal_move:
                image=get_pict()

                squares=get_squares(image) #with the camera pi

                fen_after=get_fen_from_board(squares,model_for_piece,model_for_color)

                move=get_move_from_fen(fen_before,fen_after)


                legal_move=game.make_move(move)
                if legal_move==False:
                    speak(pick_and_place(board_before))


            if len(game_sequence)>0:
                predicted_move, prediction = lstm_predict(model,tokenizer,game_sequence[-30:])
                reward,FB = compute_rl_feedback(predicted_move, str(move),cluster)
                train_lstm_with_reward(model,tokenizer,game_sequence[-30:], str(move),reward)
                comment+=FB

            game.make_move(move)
            game_sequence.append(str(move))

            rate=[0 for i in range(6)]
            comment=explain_move(fen_before,game.board.fen(), move,rate)
            game.update(rate)        

            fen_before=game.board.fen()
            
            show_board(ax, fen_before,comment)
            speak(comment)
            game.active_color = game.arm_color
        else:
            ai_move = get_move(fen_before)  

            game_sequence.append(str(ai_move))
            board_before = chess.Board(fen_before)
            piece =  board_before.piece_at(ai_move.from_square)

            comment=f"i played {piece} to {ai_move.uci()[2:]}"
            game.make_move(ai_move)
            board_before = chess.Board(fen_before)
            piece =  board_before.piece_at(ai_move.from_square)
            fen_before = game.board.fen()

            show_board(ax, fen_before,comment)
            
            game.active_color = game.student_color      
            speak(comment)    
            

     # Announce game result
    if game.board.is_checkmate():
        result="Checkmate! Game over.\n"
        if game.board.turn == chess.WHITE:
            result+="Black wins!"
            game.result="0-1"
        else:
            result+="White wins!"
            game.result="1-0"
    elif game.board.is_stalemate():
        result="Stalemate! Game over."   
        game.result="1/2-1/2"
    elif game.board.is_insufficient_material():
        result = "Draw due to insufficient material."
        game.result = "1/2-1/2"
    elif game.board.is_seventyfive_moves():
        result = "Draw due to the seventy-five-move rule."
        game.result = "1/2-1/2"
    elif game.board.is_fivefold_repetition():
        result = "Draw due to fivefold repetition."
        game.result = "1/2-1/2"    
    ax.set_title(result, fontsize=12, pad=10)
    plt.ioff()
    plt.show()#to keep the screen    
        

    # save_game_to_pgn(game, filename="output_games.pgn")
    return game

game=play()


<>:42: SyntaxWarning: invalid escape sequence '\c'
<>:47: SyntaxWarning: invalid escape sequence '\c'
<>:42: SyntaxWarning: invalid escape sequence '\c'
<>:47: SyntaxWarning: invalid escape sequence '\c'
C:\Users\yassi\AppData\Local\Temp\ipykernel_7884\1712980086.py:42: SyntaxWarning: invalid escape sequence '\c'
  model_for_piece = torch.load('models\chess_piece_classifier.pth')  # Load the piece classification model
C:\Users\yassi\AppData\Local\Temp\ipykernel_7884\1712980086.py:47: SyntaxWarning: invalid escape sequence '\c'
  model_for_color = torch.load('models\chess_color_classifier.pth')  # Load the color classification model


i played P to e4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
You moved your P from c7 to c5. Good move! Controlling the center with pawns is a key opening principle.
i played N to f3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
You moved your P from d7 to d6. This move controls the center, a key strategy in chess. Good move!
i played P to d4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
You moved your P from c5 to d4. This move controls the center, a key strategy in chess. Good move! You've played 1. d4, leading to closed games like the Queen's Gambit or King's Indian Defense.
i played N to d4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
You moved your N from g8 to f6. Knights are tricky pieces. Consider whether this move creates a tactical advantage, like a fork. Good move! Developing your minor pieces (knights and bishops) early helps control the center and prepares for castling.
i played N to c3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


In [2]:
id_value, name, level,cluster,close= show_id_manager_interface()
if not close:
    while id_value==None:
        id_value, name, level,close= show_id_manager_interface()
        if close==True:
            break
    
    print(f"ID: {id_value}, Name: {name}, Level: {level}")

    replay=True
    while replay:
        game=play(name,id_value,level,cluster)
        game.save_to_pgn("game.pgn")
        # replay=True
        replay=ask_to_play_another()  
os._exit(0)